In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline 

path_to_data = 'data/'

df = pd.read_csv(path_to_data +'comp_dates.csv')

#### Create an empty df with composerName ('last name, first name'), birthYear, deathYear

In [2]:
df = df.rename(columns = {"Unnamed: 0": "composerName"})
df['nationality'] = ""
df = df.set_index("composerName")

### Use Wikipedia API to determine composers' nationality

In [3]:
import wikipediaapi
import re

wiki_wiki = wikipediaapi.Wikipedia('en')

nationalities = set(['American', "French", "German", "Austrian",
                    "Polish", "Norwegian", "Swedish", "Finnish", "Danish", "Russian", 
                    "English", "British", "Italian", "Spanish", "Portuguese",
                   "Brazilian", "Mexican", "Czech", "Soviet", "Hungarian", 
                     "Canadian", "Chinese", "Persian", "Japanese", "Korean", "Bohemian",
                    "Greek", "Belgian", "Australian", "Dutch", "Estonian", "Romanian", "Iceland"])

In [4]:
not_found = [] #list of not found composers
na = []

for i in range(0, len(df)):#len(df)): #request times out occasionally, so picking up where it stopped
    
    # transforming from 'Beethoven, Ludwig van' to 'Ludwig van Beethoven'
    composer = df.index[i]
    names = re.split(r'[,]', composer) 
    names.reverse() 
    composer = " ".join(names)
    
    #some composers have alternate spellings in their name between []. Remove them to facilitate Wikipedia lookup.
    if "[" in composer:
        i_i = composer.find('[') -1
        j_j = composer.find(']') + 1
        composer = composer.replace(composer[i_i:j_j], "")
        
    comp_pg = wiki_wiki.page(composer)
    
    if comp_pg.exists():
        summary = wiki_wiki.extracts(comp_pg, exsentences=2)
        summary_list = re.findall(r"[\w']+|[.,!?;]", summary)
        nationality = list(nationalities.intersection(summary_list))
        
        if nationality:
            if nationality[0] == "British":
                nationality[0] = "English"
            elif nationality[0] == "Soviet":
                nationality[0] = "Russian"
            elif nationality[0] == "Canadian" or nationality[0] == "American":
                nationality[0] = "North American"
            elif nationality[0] == "German" or nationality[0] == "Austrian" or nationality[0] == "Bohemian":
                nationality[0] = "Austro-German"
            elif nationality[0] == "Swedish" or nationality[0] == "Norwegian" or nationality[0] == "Finnish":
                nationality[0] = "Nordic"
            elif nationality[0] == "Belgian":
                nationality[0] = "French"
                
            df["nationality"][i] = nationality[0]
        
        else:
            na.append(composer)

    else:
        df["nationality"][i] = "NOT FOUND"
        not_found.append(composer)

/Users/rogerioshieh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/rogerioshieh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


##### Manually inputing first 100 composers.

In [5]:
for i in range(100):
    if not df['nationality'][i]:
        df['nationality'][i] = input(f"The nationality of {df.index[i]} is")
                                    

The nationality of Mozart,  Wolfgang  Amadeus isAustro-German


/Users/rogerioshieh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


The nationality of Traditional, is
The nationality of Anthem, is
The nationality of Franck,  César isFrench
The nationality of Humperdinck,  Engelbert isAustro-German


#### Saving to file

In [6]:
df.to_csv(path_to_data + "comp_dates_nationality.csv")

f = open(path_to_data + "not-available_nationality.txt","w+")

for comp in na:
    f.write(comp + "\n")
    
f.close()